In [42]:
import os
import requests
import json
import pandas as pd
from sec_api import QueryApi

In [52]:
%env SEC_API="API_KEY"

env: SEC_API="API_KEY"


In [44]:
# convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                values.append(fact['value'])
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                indicies.append(index)                    

        income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 

    income_statement = pd.DataFrame(income_statement_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return income_statement.T 

In [51]:
# Inits QueryApi Session
queryApi = QueryApi(api_key=os.environ.get('SEC_API'))

# Fetches filing URL for Company 10-Q
query = {
  "query": { "query_string": { 
      "query": "ticker:AAPL AND filedAt:{2020-01-01 TO 2021-12-31} AND formType:\"10-Q\"" 
    } },
  "from": "0",
  "size": "10",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

filings = queryApi.get_filings(query)

print(filings)

{'total': {'value': 5, 'relation': 'eq'}, 'query': {'from': 0, 'size': 10}, 'filings': [{'id': 'c34da5ee69fe7b95fbef9365b0d4e9b2', 'accessionNo': '0000320193-21-000056', 'cik': '320193', 'ticker': 'AAPL', 'companyName': 'Apple Inc.', 'companyNameLong': 'Apple Inc. (Filer)', 'formType': '10-Q', 'description': 'Form 10-Q - Quarterly report [Sections 13 or 15(d)]', 'filedAt': '2021-04-28T18:02:54-04:00', 'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/320193/000032019321000056/0000320193-21-000056.txt', 'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/320193/000032019321000056/0000320193-21-000056-index.htm', 'linkToXbrl': '', 'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/320193/000032019321000056/aapl-20210327.htm', 'entities': [{'companyName': 'Apple Inc. (Filer)', 'cik': '320193', 'irsNo': '942404110', 'stateOfIncorporation': 'CA', 'fiscalYearEnd': '0925', 'type': '10-Q', 'act': '34', 'fileNo': '001-36743', 'filmNo': '21866148', 'sic': '3571 Electronic Com

In [47]:
# 10-Q filing URL of Apple
filing_url = 'https://www.sec.gov/Archives/edgar/data/320193/000032019321000056/aapl-20210327.htm'

# API endpoint
xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"

# get your API key at https://sec-api.io
api_key = os.environ.get('SEC_API')

final_url = xbrl_converter_api_endpoint + "?htm-url=" + filing_url + "&token=" + api_key

# make request to the API
response = requests.get(final_url)

# load JSON into memory
xbrl_json = json.loads(response.text)

# income statement example
income_statement = get_income_statement(xbrl_json)

income_statement

,2020-12-27-2021-03-27,2019-12-29-2020-03-28,2020-09-27-2021-03-27,2019-09-29-2020-03-28
RevenueFromContractWithCustomerExcludingAssessedTax,89584000000,58313000000,201023000000,150132000000
CostOfGoodsAndServicesSold,51505000000,35943000000,118616000000,92545000000
GrossProfit,38079000000,22370000000,82407000000,57587000000
ResearchAndDevelopmentExpense,5262000000,4565000000,10425000000,9016000000
SellingGeneralAndAdministrativeExpense,5314000000,4952000000,10945000000,10149000000
OperatingExpenses,10576000000,9517000000,21370000000,19165000000
OperatingIncomeLoss,27503000000,12853000000,61037000000,38422000000
NonoperatingIncomeExpense,508000000,282000000,553000000,631000000
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,28011000000,13135000000,61590000000,39053000000
IncomeTaxExpenseBenefit,4381000000,1886000000,9205000000,5568000000
